In [2]:
import numpy as np
import torch 
from torch import nn,optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices,char_to_idx,idx_to_char,vocab_size)=d2l.load_data_jay_lyrics()

In [16]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
    
    W_xi,W_hi,b_i=_three()
    W_xf,W_hf,b_f=_three()
    W_xo,W_ho,b_o=_three()
    W_xc,W_hc,b_c=_three()
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

will use cuda


In [17]:
def init_lstm_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),
           torch.zeros((batch_size,num_hiddens),device=device))

In [18]:
def lstm(inputs,state,params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H,C)=state
    outputs=[]
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,C)

In [19]:
num_epochs,num_steps,batch_size,lr,clipping_theta=160,35,32,1e2,1e-2
pred_period,pred_len,prefixes=40,50,['分开','不分开']



In [20]:
d2l.train_and_predict_rnn(lstm,get_params,init_lstm_state,num_hiddens,
                         vocab_size,device,corpus_indices,idx_to_char,
                         char_to_idx,False,num_epochs,num_steps,lr,
                         clipping_theta,batch_size,pred_period,pred_len,
                         prefixes)

epoch 40,perplexity 207.835457,time 1.93 sec
 - 分开 我不不我你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你 
 - 不分开 我不不我你你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你 我不不我你你
epoch 80,perplexity 65.408312,time 2.06 sec
 - 分开 我想你你想你 我想要你不不 我不要我 我不要我 我不要我 我不要我 我不要我 我不要我 我不要我 
 - 不分开 我想你你想你 我不要你 我不要我 我不要我 我不要我 我不要我 我不要我 我不要我 我不要我 我不
epoch 120,perplexity 14.780714,time 1.83 sec
 - 分开 我想你你的微笑 我想想这样 我想这这样活 我想要这生活 我爱你 你爱我 我想好这生活 你知后觉 你
 - 不分开 你知到我不经 我想要这样 我爱经这节活 后知后觉 你已了我 我不好好 你你怎么 你不了空 你不了空
epoch 160,perplexity 3.910669,time 1.68 sec
 - 分开 我已那的话笑笑天的老望 我有我能爸你在一个的诗 我有在美索后后想的可原 我有你世字写 想你想不不不
 - 不分开你的画面 我想要这样 如果要发发信信命运 感谢开不口 让让一名名动动动 我以以让 你子一外在半抱 我


In [ ]:
lr=1e-2
lstm_layer=nn.LSTM(input_size=vocab_size,hidden_size=num_hiddens)
model=d2l.RNNModel(lstm_layer,vocab_size)
d2l.train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,
                                 corpus_indices,idx_to_char,char_to_idx,
                                 num_epochs,num_steps,lr,clipping_theta,
                                 batch_size,pred_period,pred_len,prefixes)

epoch 40,perplexity 1.021683,time 0.30 sec
- 分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
- 不分开 我不再再想 我不要再想 我不 我不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我
